# 사용하는 라이브러리 import

In [22]:
import mysql.connector
import pandas as pd
from mysql.connector import errorcode
from __future__ import print_function
from datetime import date, datetime, timedelta
import json

# mysql 연결

In [2]:
host = "127.0.0.1"
user="root"
passwd="wilfriedzaha11!"

In [3]:
try:
    connection = mysql.connector.connect(
        host=host,
        user=user,
        passwd=passwd
    )
    
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)


In [4]:
connection.get_server_info()

'8.0.26'

# 데이터불러오기

In [5]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

# 데이터베이스 생성

In [6]:
# 데이터베이스 진입을 위한 cursor 생성
cursor = connection.cursor()
DB_NAME = 'melondb'
TABLES = {}

## 데이터 베이스를 새로 생성하는 함수

In [7]:
def create_database(cursor,DB_NAME):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME)
        )
    except mysql.connector.Error as err:
        print("데이터베이스를 생성하는데 실패했습니다.: {}".format(err))

In [8]:
# USE melondb를 한번 실행해보고

try:
    cursor.execute("USE {}".format(DB_NAME))
    

# 안되면 만들던지 아니면 종료    
except mysql.connector.Error as err:
    print("{} 데이터베이스는 존재하지 않습니다.".format(DB_NAME))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor,DB_NAME)
        print("{} 데이터베이스가 성공적으로 생성되었습니다.".format(DB_NAME))
        connection.database = DB_NAME
    else:
        print(err)
        exit(1)

# 플레이리스트, 태그 테이블

## 플레이리스트 테이블 정의

In [9]:
TABLES['Playlist'] = (
    "CREATE TABLE `Playlist` ("
    "  `playlist_id` int,"
    "  `playlist_title` varchar(255) NOT NULL,"
    "  `update_date` date NOT NULL,"
    "  `like_cnt` int NOT NULL,"
    "  PRIMARY KEY (`playlist_id`)"
    ")")

## 태그 테이블 정의

In [10]:
TABLES['Tag'] = (
    "CREATE TABLE `Tag` ("
    "  `tag_id` int,"
    "  `tag` varchar(255) NOT NULL,"
    "  PRIMARY KEY (`tag_id`)"
    ")")

## 플레이리스트,태그 관계 테이블 정의

In [11]:
TABLES['Playlist_Tag'] = (
    "CREATE TABLE `Playlist_Tag` ("
    "  `tag_id` int NOT NULL,"
    "  `playlist_id` int NOT NULL,"
    " CONSTRAINT FOREIGN KEY (`tag_id`)"
    "  REFERENCES `Tag` (`tag_id`) ON DELETE CASCADE,"
    " CONSTRAINT FOREIGN KEY (`playlist_id`)"
    "  REFERENCES `Playlist` (`playlist_id`) ON DELETE CASCADE"
    ")")

# 유저 테이블

## 유저 테이블 정의

In [12]:
TABLES['User'] = (
    "CREATE TABLE `User` ("
    "  `user_id` int NOT NULL,"
    "  `user_name` varchar(255) NOT NULL,"
    "  `playlist_id` int NOT NULL,"
    "  PRIMARY KEY (`user_id`),"
    " CONSTRAINT `playlist_fk1` FOREIGN KEY (`playlist_id`)"
    "  REFERENCES `Playlist` (`playlist_id`) ON DELETE CASCADE"
    ")")

# 노래,장르 테이블

## 노래 테이블 정의

In [13]:
TABLES['Song'] = (
    "CREATE TABLE `Song` ("
    "  `song_id` int NOT NULL,"
    "  `song_name` varchar(255) NOT NULL,"
    "  `issue_date` date NOT NULL,"
    "  `album_name` varchar(255) NOT NULL,"
    "  `album_id` int NOT NULL,"
    "  `added_cnt` int NOT NULL,"
    "  `thumb_url` varchar(255),"
    "  PRIMARY KEY (`song_id`)"      
    ")")

## 플레이리스트,노래 관계 테이블 정의

In [14]:
TABLES['Playlist_Song'] = (
    "CREATE TABLE `Playlist_Song` ("
    "  `song_id` int NOT NULL,"
    "  `playlist_id` int NOT NULL,"
    " CONSTRAINT FOREIGN KEY (`song_id`)"
    "  REFERENCES `Song` (`song_id`) ON DELETE CASCADE,"
    " CONSTRAINT FOREIGN KEY (`playlist_id`)"
    "  REFERENCES `Playlist` (`playlist_id`) ON DELETE CASCADE"
    ")")

## 대장르 테이블 정의

In [15]:
TABLES['Genre_Big'] = (
    "CREATE TABLE `Genre_Big` ("
    "  `genre_b_id` int NOT NULL,"
    "  `genre_b_name` varchar(255) NOT NULL,"
    "  PRIMARY KEY (`genre_b_id`)"   
    ")")

## 대장르,노래 관계 테이블 정의

In [16]:
TABLES['Song_Genre_Big'] = (
    "CREATE TABLE `Song_Genre_Big` ("
    "  `song_id` int NOT NULL,"
    "  `genre_b_id` int NOT NULL,"
    " CONSTRAINT FOREIGN KEY (`song_id`)"
    "  REFERENCES `Song` (`song_id`) ON DELETE CASCADE,"
    " CONSTRAINT FOREIGN KEY (`genre_b_id`)"
    "  REFERENCES `Genre_Big` (`genre_b_id`) ON DELETE CASCADE"
    ")")

## 세부장르 테이블 정의

In [17]:
TABLES['Genre_Detail'] = (
    "CREATE TABLE `Genre_Detail` ("
    "  `genre_d_id` int NOT NULL,"
    "  `genre_d_name` varchar(255) NOT NULL,"
    "  PRIMARY KEY (`genre_d_id`)"   
    ")")

## 세부장르,노래 관계 테이블 정의

In [18]:
TABLES['Song_Genre_Detail'] = (
    "CREATE TABLE `Song_Genre_Detail` ("
    "  `song_id` int NOT NULL,"
    "  `genre_d_id` int NOT NULL,"
    " CONSTRAINT FOREIGN KEY (`song_id`)"
    "  REFERENCES `Song` (`song_id`) ON DELETE CASCADE,"
    " CONSTRAINT FOREIGN KEY (`genre_d_id`)"
    "  REFERENCES `Genre_Detail` (`genre_d_id`) ON DELETE CASCADE"
    ")")

# 아티스트 테이블

## 아티스트 테이블 정의

In [19]:
TABLES['Artist'] = (
    "CREATE TABLE `Artist` ("
    "  `artist_id` int NOT NULL,"
    "  `artist_name` varchar(255) NOT NULL,"
    "  PRIMARY KEY (`artist_id`)"   
    ")")

## 아티스트,노래 관계 테이블 정의

In [20]:
TABLES['Artist_Song'] = (
    "CREATE TABLE `Artist_Song` ("
    "  `artist_id` int NOT NULL,"
    "  `song_id` int NOT NULL,"
    " CONSTRAINT FOREIGN KEY (`song_id`)"
    "  REFERENCES `Song` (`song_id`) ON DELETE CASCADE,"
    " CONSTRAINT FOREIGN KEY (`artist_id`)"
    "  REFERENCES `Artist` (`artist_id`) ON DELETE CASCADE"
    ")")

# MySQL DB에 테이블 생성

In [21]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print('{} 테이블 생성....'.format(table_name),end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")
        
#cursor.close()
#connection.close()

Playlist 테이블 생성....already exists.
Tag 테이블 생성....already exists.
Playlist_Tag 테이블 생성....already exists.
User 테이블 생성....OK
Song 테이블 생성....already exists.
Playlist_Song 테이블 생성....already exists.
Genre_Big 테이블 생성....already exists.
Song_Genre_Big 테이블 생성....already exists.
Genre_Detail 테이블 생성....OK
Song_Genre_Detail 테이블 생성....OK
Artist 테이블 생성....OK
Artist_Song 테이블 생성....OK


# 데이터 insert